In [431]:

# Importing all necessarry packages

import pandas as pd
import numpy as np


from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

import nltk

from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn import preprocessing


from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc

# Loading File

In [432]:

# Reading JSON file into Pandas DataFrame



def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

import json
import gzip

def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield json.dumps(eval(l))

f = open("output.strict", 'w')
for l in parse(r"C:\Users\Eswar\Desktop\Hackathon\Music.json.gz"):
    f.write(l + '\n')

import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF(r'C:\Users\Eswar\Desktop\Hackathon\Music.json.gz')

In [433]:
df_use=df[['reviewText']]  # creating df_use data frame

# Appending positive and Negetive labels based on Overall Rating

In [434]:
result = [] 
for value in df["overall"]: 
    if value == 1.0: 
        result.append("Negative") 
    elif value==2.0: 
        result.append("Negative") 
    elif value==3.0: 
        result.append("Neutral")
    elif value==4.0: 
        result.append("Positive")
    elif value==5.0: 
        result.append("Positive")
       
df_use["Result"] = result    

C:\Users\Eswar\New folder\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [435]:
df_use

,reviewText,Result
0,"Not much to write about here, but it does exac...",Positive
1,The product does exactly as it should and is q...,Positive
2,The primary job of this device is to block the...,Positive
3,Nice windscreen protects my MXL mic and preven...,Positive
4,This pop filter is great. It looks and perform...,Positive
...,...,...
10256,"Great, just as expected. Thank to all.",Positive
10257,I've been thinking about trying the Nanoweb st...,Positive
10258,I have tried coated strings in the past ( incl...,Positive
10259,"Well, MADE by Elixir and DEVELOPED with Taylor...",Positive


# Dropping Neutral results

In [436]:
df_use=df_use.loc[df_use['Result']!='Neutral']

# Statistics of Data Set

In [437]:
df_use.shape

(9489, 2)

# Upsampling to increase negetive samples

In [438]:
from sklearn.utils import resample
y = df_use.Result
X = df_use.drop('Result', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)
X = pd.concat([X_train, y_train], axis=1)
not_fraud = X[X.Result=='Positive']
fraud = X[X.Result=='Negative']
fraud_upsampled = resample(fraud,replace=True,n_samples=len(not_fraud), random_state=27) 
upsampled = pd.concat([not_fraud, fraud_upsampled])
#upsampled.Result.value_counts()  # To check sampled values 

In [459]:
upsampled

,reviewText,Result
728,I like the fact that it can be &#34;partially&...,Positive
5825,This thing is amazing. Like many I cannot affo...,Positive
321,"I love these strings, and use them for all of ...",Positive
7846,Love these things.I feel a little guilty becau...,Positive
2683,I have stayed clear of using pedals for most o...,Positive
...,...,...
8147,I ordered this from the Amazon warehouse as a ...,Negative
3083,All I gotta say about this product is that I ...,Negative
3987,"Cheap crappy strap. It's too stiff, not comfor...",Negative
3192,"Need 2 volume knobs and 1 tone, so for the pri...",Negative


# Splitting testing and traning data

In [439]:

X_train, X_test, y_train, y_test = train_test_split(upsampled['reviewText'], upsampled['Result'], test_size=0.25 )

In [440]:
#Converting positive / Negetive labels to 1/0

encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test  = encoder.fit_transform(y_test)

In [441]:
# Checking size of training and testing samples
print(y_test.shape)
print(y_train.shape)
print(X_test.shape)
print(X_train.shape)

(3387,)
(10159,)
(3387,)
(10159,)


In [442]:
y_test

array([0, 1, 0, ..., 1, 0, 0])

# Using TF-IDF to convert text to Vector 

In [443]:
tfidf_vect = TfidfVectorizer(analyzer='word',  token_pattern=r'\w{2,}', max_features=5000)

In [461]:
tfidf_fit_tra = tfidf_vect.fit_transform(X_train).toarray()     # Fit-transform of Train data

tfidf_fit_tra

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Applying Logistic Regression

In [468]:
logreg=LogisticRegression()

y_train=y_train.astype('int')            # converting to int

logreg.fit(tfidf_fit_tra, y_train)       #Fitting Logistic Regression on Training data

tfidf_fit_tra_test=tfidf_vect.fit_transform(X_test)   # Transforming Test data according to TF-IDF fit

y_pred = logreg.predict(tfidf_fit_tra_test)          #Predicting values on test dataset

C:\Users\Eswar\New folder\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.


In [469]:
# Checking predicted and actual

print(y_pred[80])
print(y_test[80])

0
0


# Accuracy by applying Logistic Regression

In [470]:

metrics.accuracy_score(y_test, y_pred)

0.6303513433717154

# Printing Confusion Matrix and different performance metrics

In [471]:
confusion_matrix(y_test, y_pred)

array([[1080,  616],
       [ 636, 1055]], dtype=int64)

In [472]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.64      0.63      1696
           1       0.63      0.62      0.63      1691

    accuracy                           0.63      3387
   macro avg       0.63      0.63      0.63      3387
weighted avg       0.63      0.63      0.63      3387



# Applying Multinomial Naive Bayes classifier

In [473]:
mnb = MultinomialNB()

mnb.fit(tfidf_fit_tra, y_train)

y_pred = mnb.predict(tfidf_fit_tra_test)

In [474]:
metrics.accuracy_score(y_test, y_pred)

0.5961027457927369

In [475]:
confusion_matrix(y_test, y_pred)

array([[1247,  449],
       [ 919,  772]], dtype=int64)

In [476]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.58      0.74      0.65      1696
           1       0.63      0.46      0.53      1691

    accuracy                           0.60      3387
   macro avg       0.60      0.60      0.59      3387
weighted avg       0.60      0.60      0.59      3387



# Manual Prediction

In [529]:
sentence = " very weak, worst quality"

In [530]:
tfidf_fit_tra_test1=tfidf_vect.transform([sentence])

In [531]:
print('predicted',mnb.predict(tfidf_fit_tra_test1))

predicted [0]


In [532]:
print('predicted',logreg.predict(tfidf_fit_tra_test1))

predicted [0]
